<a href="https://colab.research.google.com/github/gil612/NLP/blob/main/NLP_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text os speech)

Anothe common term for NLP problems is `seq2seq` problems.


# Connect to GPU

In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-10-15 19:32:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-10-15 19:32:54 (81.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2023-10-15 19:32:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.207, 108.177.11.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-10-15 19:32:59 (103 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# New Section

In [ ]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
len(train_df), len(test_df)

(7613, 3263)

In [ ]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [ ]:
import random
random_index = random.randint(0,len(train_df) - 5)
# Create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index: random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")



Target: 1 (real disaster)
Text:
Zeal is a volcano the peak of which the grass of indecisiveness does not grow

---

Target: 0 (not real disaster)
Text:
you can stop panicking ?????? @ogtomd  https://t.co/ZvRE6fFNyD

---

Target: 0 (not real disaster)
Text:
#Spotlight Take Me To Paradise by Arsonist MC #WNIAGospel http://t.co/1he4UfaWZm @arsonistmusic http://t.co/BNhtxAEZMM

---

Target: 1 (real disaster)
Text:
CHPSRE: RT: Refugees: For our followers in Paris do visit 'A Dream of Humanity' exhibition by rezaphotography Û_ Û_ http://t.co/RPmTROPsVr

---

Target: 0 (not real disaster)
Text:
'There is no victory at bargain basement prices.' Dwight David Eisenhower

---



In [ ]:
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


## Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Convert text into number

When dea;ingwith a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature  vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization # after TensorFlow 2.6

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [ ]:
# train_sentences[0]

twt = "@mogacola @zamtriossu i screamed after hitting tweet"
twt.split(), len(twt.split())


(['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'], 7)

In [ ]:
# Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
## adapt()
# Computes a vocabulary of string terms from tokens in a dataset.

# Calling adapt() on a TextVectorization layer is an alternative to
# passing in a precomputed vocabulary on construction via the vocabulary
# argument. A TextVectorization layer should always be either adapted
# over a dataset or supplied with a vocabulary.

# During adapt(), the layer will build a vocabulary of all string tokens
# seen in the dataset, sorted by occurrence count, with ties broken by
# sort order of the tokens (high to low). At the end of adapt(), if
# max_tokens is set, the vocabulary wil be truncated to max_tokens
# size. For example, adapting a layer with max_tokens=1000 will compute
# the 1000 most frequent tokens occurring in the input dataset. If
# output_mode='tf-idf', adapt() will also learn the document
# frequencies of each token in the input dataset.

# In order to make TextVectorization efficient in any distribution
# context, the vocabulary is kept static with respect to any compiled
# tf.Graphs that call the layer. As a consequence, if the layer is
# adapted a second time, any models using the layer should be re-compiled.
# For more information see
# tf.keras.layers.experimental.preprocessing.PreprocessingLayer.adapt.

# adapt() is meant only as a single machine utility to compute layer
# state.  To analyze a dataset that cannot fit on a single machine, see
# [Tensorflow Transform](
# https://www.tensorflow.org/tfx/transform/get_started) for a
# multi-machine, map-reduce solution.

# Arguments:
#   data: The data to train on. It can be passed either as a
#       tf.data.Dataset, or as a numpy array.
#   batch_size: Integer or None.
#       Number of samples per state update.
# If unspecified, batch_size will default to 32.
# Do not specify the batch_size if your data is in the
# form of datasets, generators, or keras.utils.Sequence instances
# (since they generate batches).
#   steps: Integer or None.
#       Total number of steps (batches of samples)
# When training with input tensors such as
# TensorFlow data tensors, the default None is equal to
# the number of samples in your dataset divided by
# the batch size, or 1 if that cannot be determined. If x is a
# tf.data dataset, and 'steps' is None, the epoch will run until
# the input dataset is exhausted. When passing an infinitely
# repeating dataset, you must specify the steps argument. This
# argument is not supported with array inputs.


# fit the text vectorizer
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
import random

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@Meganbee92 @kadiegrr im just devastated that when  it ends I will no longer see tyler blackburns face on pll xxx      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    1,   32,   29,  901,   16,   45,   15, 2475,    8,   38,
          40,  600,   99, 4355]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words in vocab: {top_5_words}")
print(f"5 least common words in vocab: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
5 least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

### Creating an Embedding using an Embedding Layer

To make out embedding, we're going to use TF's layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

Thhe parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1")

embedding

In [ ]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Remember how Nora Jones crushed it in Two Weeks Notice?      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03680153,  0.00760698, -0.03485399, ...,  0.04827707,
         -0.02673428, -0.04491077],
        [ 0.01655251, -0.04029074,  0.00352174, ..., -0.03804467,
         -0.00872298,  0.00354398],
        [-0.02800102, -0.00723401,  0.01297263, ...,  0.00929561,
          0.03312293, -0.02469927],
        ...,
        [ 0.01131805, -0.02221109, -0.04473823, ..., -0.02907091,
          0.03421256,  0.023281  ],
        [ 0.01131805, -0.02221109, -0.04473823, ..., -0.02907091,
          0.03421256,  0.023281  ],
        [ 0.01131805, -0.02221109, -0.04473823, ..., -0.02907091,
          0.03421256,  0.023281  ]]], dtype=float32)>

In [ ]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03680153,  0.00760698, -0.03485399,  0.00748708, -0.03060555,
         0.03595329,  0.01130092,  0.01087381,  0.02231734,  0.0236562 ,
         0.044624  , -0.01797645, -0.01374726,  0.00036265, -0.00511355,
         0.00125486,  0.02471929, -0.01030761, -0.00308432, -0.01233281,
         0.0171621 , -0.03039106, -0.04129963, -0.03130648,  0.03032765,
         0.04280144,  0.01530491, -0.03054042, -0.02904824,  0.01730954,
         0.04466529, -0.02725291,  0.00649544, -0.03103168, -0.03794529,
         0.01660376, -0.00643246,  0.01755089, -0.00695958,  0.01060838,
        -0.00754394, -0.03274496,  0.00265091,  0.00449389,  0.03710349,
         0.01941954, -0.00787523,  0.03065815,  0.04893663,  0.02359438,
        -0.02808166, -0.03332148, -0.04069898,  0.02356083,  0.01656893,
         0.00959033, -0.03400689, -0.04370023,  0.01554647,  0.00661794,
        -0.04100844, -0.03945633, -0.02896174, -0.00229882, -0.01843113,
  

In [ ]:
# Text vectorization / toeknization is converting words to some numrical format

## Modelling and text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series od modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed_forward nueral netwerok (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirictional-LSTM model (RNN)
* Modle 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (using Transfer Learning TL for NLP)
* Model 7: Same as model 6 with 10% of trainig data

How are we going to approach all of these?

Use standard steps in modelling with TF:
* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machimne learning modelling experiments, it's important ro create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baselin, we'l use Sklearn;s Multinomial NB using the TF-IDF formula to convert our words to numbers.

> **Note:** It's a common practice to use non-DL algorithms as baseline because of their speed and then later using their DL to see if you can improve upon them.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [ ]:
train_df.value_counts()

id     keyword   location                        text                                                                                                                                 target
48     ablaze    Birmingham                      @bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C                                                                              1         1
7166   mudslide  Chiswick, London                2 great new recipes; mudslide cake and so sorry stew! #GBBO                                                                          0         1
7178   mudslide  Edinburgh                       @hazelannmac ooh now I feel guilty about wishing hatman out. I bet the mudslide was delicious!                                       0         1
7176   mudslide  London                          First impressions: glad hat man is leaving in lieu of more interesting ladies. Hope mudslide lady triumphs next week.                0         1
7174   mudslide  The Pumpkin Carria

In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however this will be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense Model


In [ ]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [ ]:
# # Build model with the Functional API
# from tensorflow.keras import layers
# inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
# x = text_vectorizer(inputs) # turn the input text into numbers
# x = embedding(x) # create an embedding of the numerized numbers
# #x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
# outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
# model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [ ]:
# Get a summary of the model
# model_1.summary()

# Once it goes from a text vectorization to embedding every token, so all 15 tokens get represened as a 128 long feature vector,
# and then we pass this embedding through our final dense layer, which outputs the size of 1.

In [ ]:
# # Compile model
# model_1.compile(loss="binary_crossentropy",
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=["accuracy"])

In [ ]:
# # Fit the model
# model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
#                               train_labels,
#                               epochs=5,
#                               validation_data=(val_sentences, val_labels),
#                               callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
#                                                                      experiment_name="simple_dense_model")])

In [ ]:
# # Check the results
# model_1.evaluate(val_sentences, val_labels)

In [ ]:
# #Make some predictions and evaluate those
# model_1_pred_probs = model_1.predict(val_sentences)
# model_1_pred_probs.shape

# # It's outputing prediction probability for each token -> poor results

In [ ]:
# model_1_pred_probs[0]

In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [ ]:
# Get a summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [ ]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20231015-193308
Epoch 1/5
215/215 [==============================] - 26s 77ms/step - loss: 0.6087 - accuracy: 0.6957 - val_loss: 0.5352 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4410 - accuracy: 0.8192 - val_loss: 0.4687 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3463 - accuracy: 0.8606 - val_loss: 0.4584 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2847 - accuracy: 0.8918 - val_loss: 0.4634 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2378 - accuracy: 0.9118 - val_loss: 0.4757 - val_accuracy: 0.7861


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4757 - accuracy: 0.7861


[0.47573575377464294, 0.7860892415046692]

In [ ]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[-0.00169164, -0.03440502, -0.0579404 , ..., -0.01863685,
          0.04899766,  0.03772494],
        [-0.03355762, -0.01322942,  0.00670521, ...,  0.01386218,
          0.04043607, -0.01751232],
        [ 0.00592961, -0.04945339, -0.00521464, ...,  0.01533494,
         -0.01196182, -0.00926164],
        ...,
        [-0.00206165, -0.04795597, -0.01315882, ...,  0.03688076,
          0.00520448,  0.01764179],
        [ 0.00150027, -0.0579044 , -0.00611616, ...,  0.01210348,
         -0.00468962,  0.07424559],
        [-0.02918409, -0.11668723, -0.03957485, ...,  0.05101632,
          0.03879667,  0.05337527]], dtype=float32)>]

In [ ]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [ ]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only

24/24 [==============================] - 0s 2ms/step


array([[0.40377584],
       [0.7448214 ],
       [0.9978064 ],
       [0.11118074],
       [0.1091909 ],
       [0.93405753],
       [0.91780436],
       [0.9926156 ],
       [0.9695133 ],
       [0.25987145]], dtype=float32)

In [ ]:
model_1_pred_probs[0]
# Anything under threshold 0.4955 is going to get label 0

array([0.40377584], dtype=float32)

In [ ]:
model_1_pred_probs[:10]

array([[0.40377584],
       [0.7448214 ],
       [0.9978064 ],
       [0.11118074],
       [0.1091909 ],
       [0.93405753],
       [0.91780436],
       [0.9926156 ],
       [0.9695133 ],
       [0.25987145]], dtype=float32)

In [ ]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [ ]:
# convert model prediction probabilities tp label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [ ]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualizing learned embeddings

In [ ]:
# get the vocaulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [ ]:
# get the weight matrix of embedding layeer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs))
embed_Weights = model_1.get_layer("embedding_1").get_weights()
embed_Weights

[array([[-0.00169164, -0.03440502, -0.0579404 , ..., -0.01863685,
          0.04899766,  0.03772494],
        [-0.03355762, -0.01322942,  0.00670521, ...,  0.01386218,
          0.04043607, -0.01751232],
        [ 0.00592961, -0.04945339, -0.00521464, ...,  0.01533494,
         -0.01196182, -0.00926164],
        ...,
        [-0.00206165, -0.04795597, -0.01315882, ...,  0.03688076,
          0.00520448,  0.01764179],
        [ 0.00150027, -0.0579044 , -0.00611616, ...,  0.01210348,
         -0.00468962,  0.07424559],
        [-0.02918409, -0.11668723, -0.03957485, ...,  0.05101632,
          0.03879667,  0.05337527]], dtype=float32)]

In [ ]:
embed_Weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_Weights.shape)

(10000, 128)


Now we've go the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TF has a handy tool called projector

And TF also has an incredible guide on word embeddings

In [ ]:
# Create embedding  files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_Weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# download files from Colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Network (RNN)

useful for sequence data

The premise of a recurrent neural network is tp se represantation of a previous input to aid the representation of a later input.


Overview of the internals of the recurrent neural network:
MIT's sequence modelling lecture
Chris Olah LSTM
Andrej Karpathy

### Model 2:LSTM

LSTM = Long Short Term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [ ]:
# create an LSTM model

from tensorflow.keras import layers


inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units = 64, return_sequences = True) (x) # whe you're stacking RNN together, you need to set return_seqences = True
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation = "relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation ="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name = "model_s_LSTM")

In [ ]:
# Get a summary
model_2.summary()

Model: "model_s_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Compile th emodel
model_2.compile(loss = "binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback (SAVE_DIR,
                                                                        "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20231015-193346
Epoch 1/5
215/215 [==============================] - 15s 55ms/step - loss: 0.2233 - accuracy: 0.9175 - val_loss: 0.5391 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1613 - accuracy: 0.9419 - val_loss: 0.5996 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1273 - accuracy: 0.9527 - val_loss: 0.7324 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1065 - accuracy: 0.9604 - val_loss: 0.8236 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0831 - accuracy: 0.9661 - val_loss: 0.9509 - val_accuracy: 0.7822


In [ ]:
# Make preidctions with LSTM model
model_2_preds_probs = model_2.predict(val_sentences)
model_2_preds_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[5.5609625e-03],
       [6.5982276e-01],
       [9.9956483e-01],
       [3.7270200e-02],
       [2.3573563e-04],
       [9.9765682e-01],
       [8.7501746e-01],
       [9.9973100e-01],
       [9.9955183e-01],
       [5.0862354e-01]], dtype=float32)

In [ ]:
# convert model 2 pred probs tp labels
model_2_preds = tf.squeeze (tf.round(model_2_preds_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [ ]:
# Calculate model 2 results
model_2_results = calculate_results (y_true = val_labels,
                                     y_pred = model_2_preds)
model_2_results

{'accuracy': 78.21522309711287,
 'precision': 0.7843528649827862,
 'recall': 0.7821522309711286,
 'f1': 0.7800522271093588}

### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent Unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Get a summary of the GRU model
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_18 (GRU)                (None, 64)                37248     
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

In [ ]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

Saving TensorBoard log files to: model_logs/GRU/20231015-202100
Epoch 1/5
215/215 [==============================] - 15s 57ms/step - loss: 0.5317 - accuracy: 0.7199 - val_loss: 0.4585 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3216 - accuracy: 0.8678 - val_loss: 0.4899 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2177 - accuracy: 0.9183 - val_loss: 0.5607 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1545 - accuracy: 0.9460 - val_loss: 0.6257 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1175 - accuracy: 0.9588 - val_loss: 0.6032 - val_accuracy: 0.7730


In [ ]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


((762, 1),
 array([[0.3408791 ],
        [0.9157338 ],
        [0.99751055],
        [0.1749926 ],
        [0.01320716],
        [0.98949105],
        [0.8068275 ],
        [0.9965391 ],
        [0.9946403 ],
        [0.42524254]], dtype=float32))

In [ ]:
# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.29658792650919,
 'precision': 0.7729426011334698,
 'recall': 0.7729658792650919,
 'f1': 0.7719692225160549}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however,  a bidirectional RNN goes from right to left as well as left to right.

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a bidirectional RNN in Tensorflow
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x=model_4_embedding(x)
# 128 because we're going for 2 directions
# x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_Bidirectional")

(None, 15, 128)


In [ ]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Get a summary
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_24 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable par

In [ ]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback (SAVE_DIR,
                                                                        "model_4_Bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_Bidirectional/20231015-204116
Epoch 1/5
215/215 [==============================] - 16s 56ms/step - loss: 0.5220 - accuracy: 0.7314 - val_loss: 0.4576 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.3104 - accuracy: 0.8716 - val_loss: 0.4930 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1969 - accuracy: 0.9238 - val_loss: 0.5520 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1277 - accuracy: 0.9574 - val_loss: 0.6437 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0952 - accuracy: 0.9688 - val_loss: 0.6981 - val_accuracy: 0.7625


In [ ]:
# Make predictions with our bidirectional model
model_4_preds= model_4.predict(val_sentences)
model_4_preds[:10]

24/24 [==============================] - 1s 4ms/step


array([[0.17027551],
       [0.7318365 ],
       [0.9989184 ],
       [0.09627782],
       [0.00206342],
       [0.9990453 ],
       [0.7906026 ],
       [0.99971956],
       [0.99947673],
       [0.12899218]], dtype=float32)

In [ ]:
# convert model 2 pred probs tp labels
# Without conversion we get ValueError: Classification metrics can't handle a mix of binary and continuous targets
model_4_preds = tf.squeeze (tf.round(model_4_preds))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate model_4 metrics
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 76.24671916010499,
 'precision': 0.7626600366229389,
 'recall': 0.7624671916010499,
 'f1': 0.7611077640788777}

## Convolutional Neural Netweorks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height x width) ...however, our text data is 1D.

Previously we've Conv2D for our image data but now we're going to use Conv1D.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs(text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

In [ ]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence nto embedding
conv_1d = layers.Conv1D(filters=64,
                        kernel_size = 5, # This is also referred to as an ngram of 5 (meaning it looks art 5 words at a time)
                        strides = 1, # default
                        activation = "relu",
                        padding = "valid") # default = "valid", the output is smaller than the output shape, "same" means output is same shape as input
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1D layer
max_pool = layers.GlobalAveragePooling1D()
# convert it to one feature of 32
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get th efeature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 64]), TensorShape([1, 64]))

In [ ]:
#embedding_test

In [ ]:
# conv_1d_output

In [ ]:
# max_pool_output

In [ ]:
# Create 1D convultional layer to model sequences
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="Conv1D")

# Build a bidirectional RNN in Tensorflow
inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x=model_5_embedding(x)
x = layers.Conv1D(filters = 64, kernel_size = 5, strides = 1, activation = "relu", padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name = "model_5_Conv1D")


In [ ]:
# Compile
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Get a summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 Conv1D (Embedding)          (None, 15, 128)           1280000   
                                                                 
 conv1d_18 (Conv1D)          (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_8 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_29 (Dense)            (None, 1)              

In [ ]:
# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback (SAVE_DIR,
                                                                        "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20231016-000648
Epoch 1/5
215/215 [==============================] - 14s 53ms/step - loss: 0.5505 - accuracy: 0.7228 - val_loss: 0.4700 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3298 - accuracy: 0.8656 - val_loss: 0.4854 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2045 - accuracy: 0.9238 - val_loss: 0.5523 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1290 - accuracy: 0.9580 - val_loss: 0.6278 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0897 - accuracy: 0.9701 - val_loss: 0.6801 - val_accuracy: 0.7927


In [ ]:
# Make predictions with our bidirectional model
model_5_preds= model_5.predict(val_sentences)
model_5_preds[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.4703825 ],
       [0.8841095 ],
       [0.9998679 ],
       [0.13600378],
       [0.00784935],
       [0.9877127 ],
       [0.74214923],
       [0.99827194],
       [0.99973077],
       [0.21512252]], dtype=float32)

In [ ]:
# convert model 2 pred probs tp labels
# Without conversion we get ValueError: Classification metrics can't handle a mix of binary and continuous targets
model_5_preds = tf.squeeze (tf.round(model_5_preds))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
# Calculate model_5 metrics
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 79.26509186351706,
 'precision': 0.7961676493734986,
 'recall': 0.7926509186351706,
 'f1': 0.7902780510246633}